In [ ]:
%%capture
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
from google.colab import userdata
api_key = userdata.get('cos_api_key')

In [ ]:
import requests
import pandas as pd

# 공통 설정
period = "M"  # 기본은 월별
start = "200001"
end = "202412"

In [ ]:
# 일반 항목 호출 함수
def get_ecos_data(stat_code, item_code, col_name, period=period, start=start, end=end):
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code}/?/?/?"
    res = requests.get(url).json()

    if 'StatisticSearch' in res and 'row' in res['StatisticSearch']:
        df = pd.DataFrame(res['StatisticSearch']['row'])

        # 평균자료만 필터링 (해당되는 경우)
        if 'ITEM_NAME2' in df.columns and '평균자료' in df['ITEM_NAME2'].values:
            df = df[df['ITEM_NAME2'] == '평균자료']

        df = df[['TIME', 'DATA_VALUE']]
        df.columns = ['날짜', col_name]
        df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m')
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        return df.set_index('날짜')
    else:
        print(f"⚠️ {col_name} 데이터 없음 또는 오류")
        return pd.DataFrame()

In [ ]:
def get_quarterly_ecos_data(stat_code, item_code, col_name, start="2000Q1", end="2024Q4"):
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/Q/{start}/{end}/{item_code}/?/?/?"
    res = requests.get(url).json()

    if "StatisticSearch" in res and "row" in res["StatisticSearch"]:
        df = pd.DataFrame(res["StatisticSearch"]["row"])

        # 평균자료만 필터링 (해당되는 경우)
        if 'ITEM_NAME2' in df.columns and '평균자료' in df['ITEM_NAME2'].values:
            df = df[df['ITEM_NAME2'] == '평균자료']

        df = df[["TIME", "DATA_VALUE"]]
        df.columns = ["날짜", col_name]

        # 분기 → 월로 바꾸기: Q1 → 01, Q2 → 04, Q3 → 07, Q4 → 10
        df["날짜"] = df["날짜"].str.replace("Q1", "01").str.replace("Q2", "04") \
                                .str.replace("Q3", "07").str.replace("Q4", "10")
        df["날짜"] = pd.to_datetime(df["날짜"], format="%Y%m")
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
        return df.set_index("날짜")
    else:
        print(f"⚠️ {col_name} 데이터 없음 또는 오류")
        return pd.DataFrame()


In [ ]:
# 월간 데이터 항목 리스트
monthly_items = [
    ("731Y006", "0000003", "원/달러 평균환율"),
    ("901Y011", "FIEE", "전체 수출 총액"),
    ("901Y011", "FIEE31", "미국 수출 총액"),
    ("901Y011", "FIEE02", "중국 수출 총액"),
    ("901Y012", "FIEF", "전체 수입 총액"),
    ("901Y012", "FIEF31", "미국 수입 총액"),
    ("901Y012", "FIEF02", "중국 수입 총액"),
    ("403Y005", "B", "소득 교역 조건 지수"),
    ("901Y060", "FF1B", "해외 직접 투자 금액"),
    ("902Y006", "KR", "한국 기준 금리"),
    ("902Y006", "US", "미국 기준 금리"),
    ("902Y006", "CN", "중국 기준 금리"),
    ("902Y008", "KR", "한국 소비자 물가지수"),
    ("902Y008", "US", "미국 소비자 물가지수"),
    ("902Y008", "CN", "중국 소비자 물가지수"),
    ("902Y014", "KR", "한국 외환 보유액"),
    ("902Y014", "US", "미국 외환 보유액"),
    ("902Y014", "CN", "중국 외환 보유액"),
    ("902Y020", "KOR", "한국 산업 생산 지수"),
    ("902Y020", "USA", "미국 산업 생산 지수"),
    ("902Y021", "KOR", "한국 실업률"),
    ("902Y021", "USA", "미국 실업률"),
    ("902Y002", "KOR", "한국 주가지수"),
    ("902Y002", "USA", "미국 주가지수"),
    ("902Y002", "CHN", "중국 주가지수"),
    ("902Y003", "010101", "WTI 유가"),
    ("902Y003", "010102", "두바이 유가"),
    ("902Y003", "010103", "Brent 유가"),
    ("902Y003", "010201", "천연가스 가격"),
    ("902Y003", "040205", "철광석 가격"),
    ("902Y003", "040202", "구리 가격"),
    ("902Y003", "040203", "알루미늄 가격"),
]

# 분기 데이터 항목 리스트
quarterly_items = [
    ("901Y061", "FF2B", "외국인 투자 금액"),
    ("902Y009", "KR", "한국 경상수지"),
    ("902Y009", "US", "미국 경상수지"),
    ("902Y009", "CN", "중국 경상수지"),
    ("902Y010", "KR", "한국 상품수지"),
    ("902Y010", "US", "미국 상품수지"),
    ("902Y010", "CN", "중국 상품수지"),
    ("902Y015", "KOR", "한국 경제성장률"),
    ("902Y015", "USA", "미국 경제성장률"),
    ("902Y015", "CHN", "중국 경제성장률"),
]

In [ ]:
import pandas as pd
import requests

# 병합 대상 데이터프레임
df_merged = pd.DataFrame()

# 1️⃣ 월간 데이터 병합
for stat_code, item_code, col_name in monthly_items:
    print(f"📥 {col_name} 불러오는 중...")
    df = get_ecos_data(stat_code, item_code, col_name)

    if df.empty:
        print(f"⚠️ {col_name} 데이터 없음")
        continue

    # 병합
    if df_merged.empty:
        df_merged = df
    else:
        df_merged = df_merged.join(df, how='left')

# 2️⃣ 분기 데이터 보간 후 병합
for stat_code, item_code, col_name in quarterly_items:
    print(f"📥 (분기) {col_name} 불러오는 중...")
    df_q = get_quarterly_ecos_data(stat_code, item_code, col_name)

    if df_q.empty:
        print(f"⚠️ {col_name} 분기 데이터 없음")
        continue

    # 분기 데이터를 월간으로 보간 (시작 월 기준/선형보간)
    df_q_monthly = df_q.resample("MS").interpolate(method="linear")

    df_merged = df_merged.join(df_q_monthly, how='left')

# 3️⃣ 결과 확인
print("✅ 병합 완료! 컬럼 수:", df_merged.shape[1])
df_merged.head()

📥 원/달러 평균환율 불러오는 중...
📥 전체 수출 총액 불러오는 중...
📥 미국 수출 총액 불러오는 중...
📥 중국 수출 총액 불러오는 중...
📥 전체 수입 총액 불러오는 중...
📥 미국 수입 총액 불러오는 중...
📥 중국 수입 총액 불러오는 중...
📥 소득 교역 조건 지수 불러오는 중...
📥 해외 직접 투자 금액 불러오는 중...
📥 한국 기준 금리 불러오는 중...
📥 미국 기준 금리 불러오는 중...
📥 중국 기준 금리 불러오는 중...
📥 한국 소비자 물가지수 불러오는 중...
📥 미국 소비자 물가지수 불러오는 중...
📥 중국 소비자 물가지수 불러오는 중...
📥 한국 외환 보유액 불러오는 중...
📥 미국 외환 보유액 불러오는 중...
📥 중국 외환 보유액 불러오는 중...
📥 한국 산업 생산 지수 불러오는 중...
📥 미국 산업 생산 지수 불러오는 중...
📥 한국 실업률 불러오는 중...
📥 미국 실업률 불러오는 중...
📥 한국 주가지수 불러오는 중...
📥 미국 주가지수 불러오는 중...
📥 중국 주가지수 불러오는 중...
📥 WTI 유가 불러오는 중...
📥 두바이 유가 불러오는 중...
📥 Brent 유가 불러오는 중...
📥 천연가스 가격 불러오는 중...
📥 철광석 가격 불러오는 중...
📥 구리 가격 불러오는 중...
📥 알루미늄 가격 불러오는 중...
📥 (분기) 외국인 투자 금액 불러오는 중...
📥 (분기) 한국 경상수지 불러오는 중...
📥 (분기) 미국 경상수지 불러오는 중...
📥 (분기) 중국 경상수지 불러오는 중...
📥 (분기) 한국 상품수지 불러오는 중...
📥 (분기) 미국 상품수지 불러오는 중...
📥 (분기) 중국 상품수지 불러오는 중...
📥 (분기) 한국 경제성장률 불러오는 중...
📥 (분기) 미국 경제성장률 불러오는 중...
📥 (분기) 중국 경제성장률 불러오는 중...
✅ 병합 완료! 컬럼 수: 42


,원/달러 평균환율,전체 수출 총액,미국 수출 총액,중국 수출 총액,전체 수입 총액,미국 수입 총액,중국 수입 총액,소득 교역 조건 지수,해외 직접 투자 금액,한국 기준 금리,...,외국인 투자 금액,한국 경상수지,미국 경상수지,중국 경상수지,한국 상품수지,미국 상품수지,중국 상품수지,한국 경제성장률,미국 경제성장률,중국 경제성장률
날짜,,,,,,,,,,,,,,,,,,,,,
2000-01-01,1130.32,12162425,2609788,1235215,12594623,2217025,1008166,37.74,244447,4.75,...,2.267485e+06,-266.900000,-84585.000000,NaN,948.700000,-98066.000000,NaN,1.995000,0.363,NaN
2000-02-01,1129.29,12676042,2554310,1273117,12023652,2241520,834291,37.17,298450,5.00,...,2.511528e+06,706.700000,-88424.666667,NaN,2244.133333,-100870.666667,NaN,1.843667,0.849,NaN
2000-03-01,1115.75,14429703,2946821,1437377,14212011,2607380,1125348,42.37,434322,5.00,...,2.755572e+06,1680.300000,-92264.333333,NaN,3539.566667,-103675.333333,NaN,1.692333,1.335,NaN
2000-04-01,1110.01,13522040,2698030,1366877,13344532,2570158,1032362,39.42,342096,5.00,...,2.999615e+06,2653.900000,-96104.000000,NaN,4835.000000,-106480.000000,NaN,1.541000,1.821,NaN
2000-05-01,1121.37,14636755,3167181,1663241,13283356,2634243,1066610,43.70,377879,5.00,...,3.574766e+06,3060.533333,-102391.000000,NaN,4983.366667,-111980.000000,NaN,1.942667,1.248,NaN


In [ ]:
df_merged.isnull().sum()

,0
원/달러 평균환율,0
전체 수출 총액,0
미국 수출 총액,0
중국 수출 총액,0
전체 수입 총액,0
미국 수입 총액,0
중국 수입 총액,0
소득 교역 조건 지수,0
해외 직접 투자 금액,0
한국 기준 금리,0


### period index 통일 & 평균자료 있을 경우만 평균 자료 불러오도록 필터링

In [ ]:
def get_ecos_data(stat_code, item_code, col_name, period=period, start=start, end=end, filter_avg=True):
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/{period}/{start}/{end}/{item_code}/?/?/?"
    res = requests.get(url).json()

    if 'StatisticSearch' in res and 'row' in res['StatisticSearch']:
        df = pd.DataFrame(res['StatisticSearch']['row'])

        if filter_avg and 'ITEM_NAME2' in df.columns and '평균자료' in df['ITEM_NAME2'].values:
            df = df[df['ITEM_NAME2'] == '평균자료']

        df = df[['TIME', 'DATA_VALUE']]
        df.columns = ['날짜', col_name]
        df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m')
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        df.set_index('날짜', inplace=True)
        df.index = df.index.to_period("M")  # ✅ PeriodIndex 통일
        return df
    else:
        print(f"⚠️ {col_name} 데이터 없음 또는 오류")
        return pd.DataFrame()


In [ ]:
def get_quarterly_ecos_data(stat_code, item_code, col_name, start="2000Q1", end="2024Q4", filter_avg=True):
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/1000/{stat_code}/Q/{start}/{end}/{item_code}/?/?/?"
    res = requests.get(url).json()

    if "StatisticSearch" in res and "row" in res["StatisticSearch"]:
        df = pd.DataFrame(res["StatisticSearch"]["row"])

        if filter_avg and 'ITEM_NAME2' in df.columns and '평균자료' in df['ITEM_NAME2'].values:
            df = df[df['ITEM_NAME2'] == '평균자료']

        df = df[["TIME", "DATA_VALUE"]]
        df.columns = ["날짜", col_name]
        df["날짜"] = df["날짜"].str.replace("Q1", "01").str.replace("Q2", "04") \
                                .str.replace("Q3", "07").str.replace("Q4", "10")
        df["날짜"] = pd.to_datetime(df["날짜"], format="%Y%m")
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
        df.set_index("날짜", inplace=True)
        df.index = df.index.to_period("M")  # ✅ PeriodIndex로 변환
        return df
    else:
        print(f"⚠️ {col_name} 데이터 없음 또는 오류")
        return pd.DataFrame()


In [ ]:
# 병합 대상 데이터프레임
df_merged = pd.DataFrame()

# 1️⃣ 월간 데이터 병합
for stat_code, item_code, col_name in monthly_items:
    print(f"📥 {col_name} 불러오는 중...")
    df = get_ecos_data(stat_code, item_code, col_name, filter_avg=True)  # 필터 사용 여부 선택 가능

    if df.empty:
        print(f"⚠️ {col_name} 데이터 없음")
        continue

    df_merged = df if df_merged.empty else df_merged.join(df, how='left')

# 2️⃣ 분기 데이터 보간 후 병합
for stat_code, item_code, col_name in quarterly_items:
    print(f"📥 (분기) {col_name} 불러오는 중...")
    df_q = get_quarterly_ecos_data(stat_code, item_code, col_name, filter_avg=True)

    if df_q.empty:
        print(f"⚠️ {col_name} 분기 데이터 없음")
        continue

    # ✅ 1. PeriodIndex → DatetimeIndex 로 변환
    df_q.index = df_q.index.to_timestamp(how="start")  # '2023Q1' → '2023-01-01'

    # ✅ 2. 월별 리샘플링 + 보간
    df_q_monthly = df_q.resample("MS").interpolate(method="linear")

    # ✅ 3. 다시 PeriodIndex로 변환 (월 단위)
    df_q_monthly.index = df_q_monthly.index.to_period("M")

    # ✅ 4. 병합
    df_merged = df_merged.join(df_q_monthly, how='left')


📥 원/달러 평균환율 불러오는 중...
📥 전체 수출 총액 불러오는 중...
📥 미국 수출 총액 불러오는 중...
📥 중국 수출 총액 불러오는 중...
📥 전체 수입 총액 불러오는 중...
📥 미국 수입 총액 불러오는 중...
📥 중국 수입 총액 불러오는 중...
📥 소득 교역 조건 지수 불러오는 중...
📥 해외 직접 투자 금액 불러오는 중...
📥 한국 기준 금리 불러오는 중...
📥 미국 기준 금리 불러오는 중...
📥 중국 기준 금리 불러오는 중...
📥 한국 소비자 물가지수 불러오는 중...
📥 미국 소비자 물가지수 불러오는 중...
📥 중국 소비자 물가지수 불러오는 중...
📥 한국 외환 보유액 불러오는 중...
📥 미국 외환 보유액 불러오는 중...
📥 중국 외환 보유액 불러오는 중...
📥 한국 산업 생산 지수 불러오는 중...
📥 미국 산업 생산 지수 불러오는 중...
📥 한국 실업률 불러오는 중...
📥 미국 실업률 불러오는 중...
📥 한국 주가지수 불러오는 중...
📥 미국 주가지수 불러오는 중...
📥 중국 주가지수 불러오는 중...
📥 WTI 유가 불러오는 중...
📥 두바이 유가 불러오는 중...
📥 Brent 유가 불러오는 중...
📥 천연가스 가격 불러오는 중...
📥 철광석 가격 불러오는 중...
📥 구리 가격 불러오는 중...
📥 알루미늄 가격 불러오는 중...
📥 (분기) 외국인 투자 금액 불러오는 중...
📥 (분기) 한국 경상수지 불러오는 중...
📥 (분기) 미국 경상수지 불러오는 중...
📥 (분기) 중국 경상수지 불러오는 중...
📥 (분기) 한국 상품수지 불러오는 중...
📥 (분기) 미국 상품수지 불러오는 중...
📥 (분기) 중국 상품수지 불러오는 중...
📥 (분기) 한국 경제성장률 불러오는 중...
📥 (분기) 미국 경제성장률 불러오는 중...
📥 (분기) 중국 경제성장률 불러오는 중...


In [ ]:
df_merged = df_merged.sort_index()

In [ ]:
df_merged.isnull().sum()

,0
원/달러 평균환율,0
전체 수출 총액,0
미국 수출 총액,0
중국 수출 총액,0
전체 수입 총액,0
미국 수입 총액,0
중국 수입 총액,0
소득 교역 조건 지수,0
해외 직접 투자 금액,0
한국 기준 금리,0
